# Walkthrough: LLM fine-tuning (Optional)

## Introduction

Have you ever wondered how an AI assistant determines the tone of a review or identifies whether feedback is positive or negative? It all starts with preprocessing—the cornerstone of any successful machine learning model.

In machine learning, preprocessing data is an essential step that directly influences your model's performance. Ensuring that your data is clean, structured, and ready for training will set the stage for more accurate predictions and better outcomes. Imagine you are preparing a dataset of patient feedback for sentiment analysis in the healthcare industry. These preprocessing steps will ensure your data is clean and ready for fine-tuning a large language model (LLM) to perform sentiment classification tasks.

This guide will take you through the essential steps in preparing datasets for machine learning tasks, focusing on text data for natural language processing (NLP) applications.

By the end of this walkthrough, you will be able to:

- Clean and preprocess text data for machine learning tasks.
- Apply tokenization, text normalization, and missing data handling techniques to ensure your data is ready for model training.
- Organize and split your dataset into appropriate training, validation, and test sets for optimal model performance.
- Create a structured workflow that prepares datasets for a variety of machine learning tasks, including fine-tuning.

## Key steps in preprocessing data for machine learning

This walkthrough will guide you through the following steps:

- Step 1: Clean text
- Step 2: Apply tokenization
- Step 3: Handle missing data
- Step 4: Normalize text
- Step 5: Prepare the data for fine-tuning
- Step 6: Split the data

## Step 1: Clean text

Text cleaning is one of the first and most important steps in preparing a dataset for machine learning. Cleaning ensures that your text data is consistent and free from unnecessary noise. This process typically involves removing special characters, URLs, and extra spaces and converting the text to lowercase to ensure uniformity. Note that the following code requires at least a passing familiarity with regular expressions. You can [read more about regular expressions in the Python documentation](https://docs.python.org/3/library/re.html).

### Example code

In [ ]:
import re
import pandas as pd

# Create a noisy sample dataset
data_dict = {
    "text": [
        " The staff was very kind and attentive to my needs!!! ",
        "The waiting time was too long, and the staff was rude. Visit us at http://hospitalreviews.com",
        "The doctor answered all my questions...but the facility was outdated. ",
        "The nurse was compassionate & made me feel comfortable!! :) ",
        "I had to wait over an hour before being seen. Unacceptable service! #frustrated",
        "The check-in process was smooth, but the doctor seemed rushed. Visit https://feedback.com",
        "Everyone I interacted with was professional and helpful. 😊 "
    ],
    "label": ["positive", "negative", "neutral", "positive", "negative", "neutral", "positive"]
}

# Convert to a DataFrame
data = pd.DataFrame(data_dict)

# Function to clean the text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

# Apply the cleaning function
data['cleaned_text'] = data['text'].apply(clean_text)
print(data[['cleaned_text', 'label']].head())

### Why is this important?

Cleaning the text ensures that the data provided to the machine learning model is consistent, removing unwanted characters or formatting that could confuse the model. Clean data leads to better feature extraction and, ultimately, improved performance during the training and testing phases. This step is particularly important when fine-tuning LLMs, as clean data ensures the model can focus on learning task-specific patterns.

## Step 2: Apply tokenization

Tokenization is the process of splitting text into individual words or tokens that a model can understand. Tokenization helps break down the text into manageable parts for analysis and learning, especially when working with transformer-based models, such as BERT.

### Example code

In [ ]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the cleaned text
def tokenize_function(text):
    return tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=128)

# Apply tokenization
data['tokenized'] = data['cleaned_text'].apply(tokenize_function)
print(data[['tokenized', 'label']].head())

### Why is this important?

Tokenization transforms raw text into a format that the machine learning model can process. Without tokenization, a model would struggle to interpret the text's meaning, particularly for NLP tasks in which understanding individual words and their contexts is critical. Using a task-specific tokenizer ensures compatibility with LLMs like BERT, which will be fine-tuned later in your workflow.

## Step 3: Handle missing data

Handling missing data is crucial to maintaining your dataset's integrity. Missing data can distort your results or cause models to misinterpret the information, leading to poor predictions. There are two primary ways to handle missing data: removing the incomplete data and filling the missing values with an appropriate placeholder or statistic (e.g., mean, median, or mode). Some of these measures of central tendency, primarily mode, for example, which counts the most common appearance of a value, are suitable for nominal data analysis.

### Example code

In [ ]:
# Check for missing data
print(data.isnull().sum())

# Option 1: Drop rows with missing data
data = data.dropna()

# Option 2: Fill missing values with a placeholder
data['cleaned_text'].fillna('missing', inplace=True)

### Why is this important?

Missing data can lead to bias in the model or cause errors during training. By addressing missing data properly, you ensure that your model learns from complete and accurate information, improving its ability to make correct predictions. This is especially crucial when preparing task-specific datasets for fine-tuning, where data quality directly impacts performance.

## Step 4: Normalize text

Text normalization refers to standardizing the format of your text to ensure consistency. This may include converting all text to lowercase, removing contractions (e.g., changing "don't" to "do not"), and correcting spelling errors. Normalization is especially important for machine learning tasks in which consistency in input data leads to better feature extraction and model performance.

### Additional considerations for normalization

- **Stemming or lemmatization:** these techniques reduce words to their base forms, which helps the model focus on the core meaning of the words rather than their specific inflected forms.
- **Removing stop words:** normalization often removes such words as "the," "is," or "and," as they do not provide significant meaning in most machine learning tasks.

## Step 5: Prepare the data for fine-tuning

After cleaning and tokenizing the text, you must prepare the data for training. In tasks like fine-tuning, structuring the data correctly ensures compatibility with LLMs like BERT. This involves organizing the tokenized data and labels into a format that the machine learning model can use during training, for example, as PyTorch DataLoader objects.

### Example code

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Prepare tensors for fine-tuning
input_ids = torch.cat([token['input_ids'] for token in data['tokenized']], dim=0)
attention_masks = torch.cat([token['attention_mask'] for token in data['tokenized']], dim=0)
labels = torch.tensor([0 if label == "negative" else 1 if label == "neutral" else 2 for label in data['label']])

# Create DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print("DataLoader created successfully!")

### Why is this important?

Structuring the data in this way ensures that the model can efficiently process it during training, allowing for smoother fine-tuning and faster convergence. By preparing your data in this format, you enable the model to handle large datasets effectively, even in real-time applications.

## Step 6: Split the data

Splitting your dataset into training, validation, and test sets is critical for ensuring your model generalizes well to unseen data. Proper data splitting allows you to monitor the model's performance during training and prevents overfitting, which occurs when the model learns patterns in the training data too well but fails to generalize.

### Example code

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training, validation, and test sets
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids, labels, test_size=0.2, random_state=42
)

# Create DataLoader objects
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

print("Data splitting successful!")

### Why is this important?

By splitting the data, you ensure that the model's performance can be evaluated on unseen data, reducing the risk of overfitting and increasing the likelihood that the model generalizes well to new data. For fine-tuning, validation data is essential for hyperparameter tuning, while test data evaluates the final model's performance. This practice is essential for building robust machine learning systems.

## Conclusion

Preprocessing data is a critical step in ensuring that your machine learning model receives high-quality input. From cleaning and tokenizing text to handling missing data and structuring it for training, each preprocessing step plays a vital role in the overall performance of your model. Properly preparing data ensures smoother fine-tuning and better model outcomes, whether you're working on NLP tasks, classification, or regression models.